In [2]:
from keras.initializers import Constant
from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Model
from keras.optimizers import adam

import numpy as np

/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Ingest training set
- training examples
- validation examples
- input parameters
    - size
    

### Set up cluster
- CPU pre-processing, GPU training
- number of workers
- threading
- number of GPUs


### Augment training set
https://keras.io/preprocessing/image/
- crop
- featurewise_center (pixel mean)
- samplewise_center (sample mean)
- flip/flop/rotate
- transpose
- scale
- methods
    - flow_from_directory


In [8]:
from keras.preprocessing import image

In [ ]:
image.ImageDataGenerator(featurewise_center=1,
                         samplewise_center=1,
                         featurewise_std_normalization=1,
                         zca_whitening=1,
                         rotation_range=60,
                         )

### Pre-process

### Set up training

- checkpoints to save models
- tensorboard
- kera ImageDataGenerator

- hyperparameters
    - batch size
    - number of epochs
    - learning rate 
    - momentum
    - optimizer
- initialization: 
    - weights https://keras.io/initializers/
    - bias can be Constant(value=0.1) for example


In [3]:
n_filters = [128, 256, 256]
kernel_size = [3, 3, 3]

X_train_shape = (224,224,3)

input_layer = Input(shape=X_train_shape)

hidden= [Conv2D(filters=90, kernel_size=11, strides=2,
                     padding='same', activation='relu', kernel_initializer='he_normal',
                     bias_initializer=Constant(value=0.1), input_shape=X_train_shape)(input_layer)]
hidden.append(MaxPool2D(strides=2)(hidden[-1]))

for nfilt, ksize  in zip(n_filters, kernel_size):
    for i in np.arange(3):
        hidden.append(Conv2D(filters=nfilt, kernel_size=ksize, padding='same', activation='relu',
                         kernel_initializer='he_normal', bias_initializer=Constant(value=0.1))(hidden[-1]))
    hidden.append(MaxPool2D(strides=2)(hidden[-1]))

n_filters = [2048, 2048, 5]
kernel_size = [7, 1, 1]

for nfilt,ksize in zip(n_filters, kernel_size):
    hidden.append(Conv2D(filters=nfilt, kernel_size=ksize, padding='same', activation='relu',
                        kernel_initializer='he_normal', bias_initializer=Constant(value=0.1))(hidden[-1]))
    hidden.append(Dropout(0.5)(hidden[-1]))

output_layer = Flatten()(hidden[-1])

In [4]:
hidden

[<tf.Tensor 'conv2d_1/Relu:0' shape=(?, 112, 112, 90) dtype=float32>,
 <tf.Tensor 'max_pooling2d_1/MaxPool:0' shape=(?, 56, 56, 90) dtype=float32>,
 <tf.Tensor 'conv2d_2/Relu:0' shape=(?, 56, 56, 128) dtype=float32>,
 <tf.Tensor 'conv2d_3/Relu:0' shape=(?, 56, 56, 128) dtype=float32>,
 <tf.Tensor 'conv2d_4/Relu:0' shape=(?, 56, 56, 128) dtype=float32>,
 <tf.Tensor 'max_pooling2d_2/MaxPool:0' shape=(?, 28, 28, 128) dtype=float32>,
 <tf.Tensor 'conv2d_5/Relu:0' shape=(?, 28, 28, 256) dtype=float32>,
 <tf.Tensor 'conv2d_6/Relu:0' shape=(?, 28, 28, 256) dtype=float32>,
 <tf.Tensor 'conv2d_7/Relu:0' shape=(?, 28, 28, 256) dtype=float32>,
 <tf.Tensor 'max_pooling2d_3/MaxPool:0' shape=(?, 14, 14, 256) dtype=float32>,
 <tf.Tensor 'conv2d_8/Relu:0' shape=(?, 14, 14, 256) dtype=float32>,
 <tf.Tensor 'conv2d_9/Relu:0' shape=(?, 14, 14, 256) dtype=float32>,
 <tf.Tensor 'conv2d_10/Relu:0' shape=(?, 14, 14, 256) dtype=float32>,
 <tf.Tensor 'max_pooling2d_4/MaxPool:0' shape=(?, 7, 7, 256) dtype=float

In [5]:
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 90)      32760     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 90)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       103808    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
__________

In [ ]:
model.compile(optimizer=adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])